#### There should be a flowchart, which illustrates what each part does, in the same folder
Note: This whole script was meant to offer seperate suggestions of "supposed to be"-results while using calculations which I am completely familiar with. There is no standard way to interpret those results. By no mean can it be implemented into the firmware (which already has functions for similiar operations).

In [1]:
#Preparation, must be run first
import math
import numpy as np

## Part I
### Change in encoder -> Change in Angle (degree)
#### Variables which needs to be known before running this programm: (i) encoder value at calibration point (ii) angle (degree) at calibration point (iii) absolute change of encoder value after movement (can also be calculated by knowing the encoder value at goal)

In [2]:
def predict_D_from_E(Ec,Dc,dE,E1=None,D1=None): #last 2 args optional
    estep=4096/360
    dstep=360/4096
    angle=["B","L","R"]
    caliplusx,caliplusy=[0,0,0],[0,0,0] #Pre-definition, to store values for the addtion later(otherwise error message)
    
    #Outputs the initial conditions just for overview
    for i in range(3):
        print("Calibration: Encoder=",angle[i],Ec[i]%4096,"  Angle=",angle[i],Dc[i]%360)
    
    #Case 1: 3 args, there is no E1 (We assume that D1 will not exist either)   
    # Ec----------E2(=caliplusx)
    # |----dD------|
    
    # Dc----------D2
    # | caliplusy |
    if E1 == None:
        print("Starting point (E1)=calibration(Ec)")
        #Calculation of E2 = Ec +step
        for i in range(3):
            caliplusx[i]=(Ec[i]%4096)+(dE[i]%4096) #=E2
            print("Final Encoder (E2): ",angle[i],caliplusx[i]%4096)
        #Turn Estep to Dstep and add Dstep on Dc
        y=[0,0,0] #placeholder to avoid error message, means dE in deg
        for i in range(3):
            #y=[0,0,0] #Current_deg= manually fetches D1 from input! #DELETE
            y[i]= (dE[i]*dstep)%360
            caliplusy[i]=(Dc[i]+y[i])%360
            print("Predicted final angle (D2): ",angle[i],caliplusy[i]%360)
    

    #Case 2: there are E1 and D1. While the firmware works slightly different, this method actually produces its result
    #NOTE: in this case, argument dE = E2-E1, instead of E2-Ec!
    #NOTE THAT THIS METHOD HEAVLY RELIES ON THE ACCURACY OF D1 WHICH YOU HAVE INPUTTED AS ARGUMEMT!

    #Ec-----------E1----------E2
    #             | caliplusx |

    #Dc-----------D1----------D2
    #             | caliplusy |
    elif E1 != None:
        for i in range(3):
            print("Starting point: Encoder val(E1):",angle[i],E1[i],"  Angle(D1):",angle[i],D1[i])
        #Calculation of E2 = E1+(E1-2)
        for i in range(3):
            caliplusx[i]=(E1[i])+(dE[i]) #calipx is a misleading variable name, E2-E1 is meant
            print("Final Encoder (E1+step): ",angle[i],caliplusx[i]%4096)
        #Turn E2-E1 into D2-D1    
        y=[0,0,0]
        for i in range(3):
            y[i]=dE[i]*dstep #y is absolute D step #This line isn't actually necessary LOL
            caliplusy[i]=D1[i]+y[i] #calculate D2= D1 + (E2-E1 as Deg)
            print("Predicted final angle (D2): ",angle[i],caliplusy[i]%360)
        
    #sigma-check: Difference between Method 1 and 2 (simply to see how much mess a wrong D1 can cause)
    for i in range(3):
        caliplusy[i] = caliplusy[i]%360 #Method 1 (D2-Dc)

        E2=caliplusx[i] #Method 2
        tD2= (((E2-Ec[i])*dstep)+Dc[i])%360 #tD2: theoretical D2, calculated from (E2-Ec)->dstep->(D2-Dc)
        sigma = tD2-(caliplusy[i])
        print("D1-dependent:",caliplusy[i],"tD2:",tD2,"Sigma: ",sigma)

#Examples
Ec=[2176,3401,2540] #encoder calibration (P2242)
Dc=[90,27.6,96.15] #angle calibration (P2200)
dE=[0,138,-269] #Changes (P2243)fin -(P2243)start
E1=[2176,3374,2755] #E1
D1=[90,25.2270,115.0465] #D1
predict_D_from_E(Ec,Dc,dE,E1,D1) #E1,D1 optional

Calibration: Encoder= B 2176   Angle= B 90
Calibration: Encoder= L 3401   Angle= L 27.6
Calibration: Encoder= R 2540   Angle= R 96.15
Starting point: Encoder val(E1): B 2176   Angle(D1): B 90
Starting point: Encoder val(E1): L 3374   Angle(D1): L 25.227
Starting point: Encoder val(E1): R 2755   Angle(D1): R 115.0465
Final Encoder (E1+step):  B 2176
Final Encoder (E1+step):  L 3512
Final Encoder (E1+step):  R 2486
Predicted final angle (D2):  B 90.0
Predicted final angle (D2):  L 37.355906250000004
Predicted final angle (D2):  R 91.403921875
D1-dependent: 90.0 tD2: 90.0 Sigma:  0.0
D1-dependent: 37.355906250000004 tD2: 37.355859375 Sigma:  -4.687500000244427e-05
D1-dependent: 91.403921875 tD2: 91.40390625 Sigma:  -1.5624999988972377e-05


## Part II
#### Previous: Turn encoder unit into angles (degree)
### Here, from 3 known angles, strech (S) and height(H) will be calculated (as mid-step) and then XYZ coordinates.\n Note that the result heavlily depends on correct S-offset.

### However, the geometry was misunderstood when this function was written(see grey arrows in Fig.1C of the report). The angle at the joint between ARM_A and ARM_B(defined as alpha in the function) might be far more complicated. 

In [11]:
#Few size-related constants to define
lA=142.07 #length ArmA
lB=158.81 #length ArmB
soff=13.2 #length center to origin

soff2=5.3 #end effector
zoff=107.4 #base of arm; z(coord)=h+zoff at the end
#s2/z offset highly depends on model!!
#s has 3 parts: soff+ the middle part used for calculating triangles + soff2 at end effector
#when calculating X and Y, the "whole" s made from 3 parts are required

#Add picture!
#//               /\          |          /              |            \
#//       ARM_A  /  \  ARM_B  |    ARM_A/               |             \ ARM_B
#//           __/    \        |        /_____  AngleA   |   angleB ____\
#//          | center         |                         |
#//     origin

In [12]:
def calc_SH_from_3angle(deglist): #also calculates XYZ
    #A lot of angles are calculated as mid-step
    alpha = 180-(deglist[1]+deglist[2])
    hypo=math.sqrt((lA**2)+(lB**2)-((2*lA*lB)*math.cos(math.radians(alpha)))) #c^2=b^2+a^2-2ab*cos(y)
    #we have LA,LB,hypo
    #now, calculate beta and delta too for other purposes
    beta= math.degrees(math.acos((-lB**2 + lA**2 + hypo**2)/(2 * lA * hypo)))#cos(ß)=b^2-a^2-c^2/-2ac
    delta= math.degrees(math.acos((lA**2 - lB**2 - hypo**2)/(-2 * lB * hypo))) 
    print("a:",alpha,"\nhypo:",hypo,"\nbeta:",beta,"\ndelta",delta,"\n")
    print("Input Angles:",deglist)
    
    s= lA*(math.cos(math.radians(deglist[1])))+lB*(math.cos(math.radians(deglist[2])))+soff+soff2 #whole S!
    h= lA*(math.sin(math.radians(deglist[1]))) - lB*(math.sin(math.radians(deglist[2])))#apparently,h is just z
    z=zoff+h #Z is good, don't touch!
    x = (math.cos(math.radians(deglist[0]-90)))*(s)
    y = (math.sin(math.radians(deglist[0]-90)))*(s)
    
    
    print("h(just z - zoff):",h,"\ns:",s,"\nX:",x,"\nY",y,"\nZ:",z)
    #Normal case: end is higher #Correction: No cases. Thought the equation would change depending on arms position
        
#For example:
caliplusy=[90,27.6,96.15] #angle B/L/R or C/A/B
calc_SH_from_3angle(caliplusy)

a: 56.25 
hypo: 142.60013293234252 
beta: 67.81746814049792 
delta 55.93253185950209 

Input Angles: [90, 27.6, 96.15]
h(just z - zoff): -92.07555416098472 
s: 127.38934859879238 
X: 127.38934859879238 
Y 0.0 
Z: 15.324445839015283


## Part III (Subpart and Mainpart)
### Subpart of III: simple subtraction between starting XYZ coordinates and final XYZ coordinates, to calculate their differences and direction
#### There is starting coord. (sc), finial coord.(fc), and difference in coord.(dc). With 2 of them, the third one can be calculated.\nIn function for linear movement (G1), the goal is also first determined from simple coordinate addition.


In [14]:
def coord_addition(sc=None,dc=None,fc=None): #only 2 required, third one =None #if loop quite unefficient
    if fc == None: #s+d
        fc=[sc[i]+dc[i] for i in range(len(sc))]
    elif dc==None: #f-s
        dc=[fc[i]-sc[i] for i in range(len(sc))]
    elif sc==None: #f-d
        dc=[fc[i]-dc[i] for i in range(len(sc))]   
    print("start:",sc,"diff:",dc,"fin:",fc)
    #Additional check: Asolute distance
    absd=math.sqrt((dc[0])**2+(dc[1])**2+(dc[2])**2)
    print("Absolute distance:",absd)
    #Additional check: Direction
    #It would be easier to just determine whether d is pos or neg, but I want to determine it through s and f
    directions=[]
    for i in range(3):
        if sc[i]<fc[i]:
            drctn="+"
        elif sc[i]>fc[i]:
            drctn="-"
        else:
            drctn="0"
        directions.append(drctn)
    print("Directions:",directions)
#The function will return 3 lists of coordinates: start, difference and fin
        
#For example
sc=[1,1,1]
dc=[10,20,-31]
fc=[1,0,-30] #NOTE: Fin is NOT s+d, for testing purpose
coord_addition(sc,dc,None)

start: [1, 1, 1] diff: [10, 20, -31] fin: [11, 21, -30]
Absolute distance: 38.22302970723278
Directions: ['+', '+', '-']


### Main part of III
### from initial and final coordinates (XYZ), calculate their corresponding angles (A/B/C)
#### 2 inputs are required for this function: list of starting coord (ls) and list of difference (ld). If they are unknown, use Part III Subpart to calculate them

In [24]:
#Already defined
lA=142.07
lB=158.81
soff=13.2 #length center to origin
soff2=5.3
zoff=107.4 #sockel
#s has 3 parts: soff+ the middle part used for calculating triangles + soff2 at end effector

#Now we want (XYZ+dXYZ) to be calculated back to degrees
def coord_addition_to_deg(ls,ld): #ls= list of XYZ start/fin, ld= list of XYZ difference, bc. s is strech here
    #first, convert XYZ start to deg(also list)
    s= (math.sqrt((ls[0]**2)+(ls[1]**2))) #full s
    if ls[1]==0: #somehow, depending what you ask the firmware, Y0-position can be 0° or 90°
        anglec=90
    else:
        anglec=math.degrees(math.atan(ls[0]/ls[1]))+90
    h=ls[2]-zoff #h can be negative!!!
    print("Initial stretch and heigh\nS:",s,"\nH: ",h)
    
    #Method 1: solve equation system for angleA,angleB by using lA,lB,s,and h
    #Roughly:
    #I)  s=(cos(A)*lA)+cos(B)*lB-s)
    #II) h=(sin(A)*lA)-(sin(B)*lB)-h)
    
    #Sympy library (python) is used!!!!!
    
    #NOTE: Following function is muted through comment, due to the high capacity required to calculate
    #NOTE: It was just an alternative to make sure that the other method is not completetly wrong
     #from sympy import Symbol, solve, Eq
        #from sympy import *
        #import math
        #Parameters, if possible, take from coord_addition_to_deg()
        #lA= 142.07
        #lB= 158.81
        #h= 15.6788
        #s= 174.1773
    #def solve_for_AB():
        #Apparently so sympy can work with variable as str?
        #A = Symbol('A',real = True)
        #B = Symbol('B',real = True)
        #Trigonometric equation I) and II)
        #e1= Eq((cos(A)*lA)+cos(B)*lB-s)
        #e2= Eq((sin(A)*lA)-(sin(B)*lB)-h)
        #print(solve([e1,e2],A,B)) #It will give 2 solution due to math reasons
        #insert relut here
        #a = math.degrees(0.0) 
        #b = math.degrees(0.0)
        #print(a,b)
        
    #Method 1 was basically used to compare and vertify Method 2
    
    #Method 2: Hypotenuse of s and h (reversed tigonometry from Part 2)
    gamma=math.degrees(math.atan(h/(s-soff-soff2))) #if h is negative, gamma will be negative too
    hypo=math.sqrt(((s-soff-soff2)**2)+(h**2)) 
    a=((lB**2)+(lA**2)-(hypo**2))/(lA*lB*2)                 
    alpha= math.degrees(math.acos(a)) #Rule of cosine, to find alpha
    a=((hypo**2)+(lA**2)-(lB**2))/(lA*hypo*2)
    beta=math.degrees(math.acos(a)) #Rule of cosine, to find beta
    anglea=beta+gamma
    angleb=180-anglea-alpha
    print("Method2, starting angles:","\nB",anglec,"L",anglea,"R",angleb)
    
    #Then, we predict the change in BLR after knowing the dXYZ as ld
    lf=[ls[0]+ld[0],ls[1]+ld[1],ls[2]+ld[2]]
    print("\nPredicted angle after dXYZ:")
    s= (math.sqrt((lf[0]**2)+(lf[1]**2))) #Full s
    if lf[1]==0: #somehow, depending what you ask the firmware, Y0-position can be 0° or 90°
        anglec=90
    else:
        anglec=math.degrees(math.atan(lf[0]/lf[1]))+90
    h=lf[2]-zoff
    print("Sf:",s,"\nHf: ",h)
    
    #Still Method 2
    gamma=math.degrees(math.atan(h/(s-soff-soff2)))#if h is negative, gamma will be negative too
    hypo=math.sqrt(((s-soff-soff2)**2)+(h**2))
    a=((lB**2)+(lA**2)-(hypo**2))/(lA*lB*2)                 
    alpha= math.degrees(math.acos(a)) #Rule of cosine, to find alpha
    a=((hypo**2)+(lA**2)-(lB**2))/(lA*hypo*2)
    beta=math.degrees(math.acos(a)) #Rule of cosine, to find beta
    anglea=beta+gamma
    angleb=180-anglea-alpha
    print("Method2, final angles:","\nB",anglec,"L",anglea,"R",angleb)


#Example
ls,ld=[30,40,60],[10,10,10] #ls= starting XYZ, ld= difference in XYZ
coord_addition_to_deg(ls,ld)

Initial stretch and heigh
S: 50.0 
H:  -47.400000000000006
Method2, starting angles: 
B 126.86989764584402 L 39.989011276180136 R 119.14729827567689

Predicted angle after dXYZ:
Sf: 64.03124237432849 
Hf:  -37.400000000000006
Method2, final angles: 
B 128.65980825409008 L 55.963040125449865 R 102.35831182532692


## Part IV
### Same principle as Part I, but we calculate the expected encoder from angles

In [25]:
#Come back to encoder
#Often, it will be a different value than encoder from P2243, as P2243 directly checks encoder, 
#while this calculates the encoder from goal coordinates
def DtoE(ec,dc,d2): #input: d2 list, ecali list, dcali list
    ang=["B","L","R"]
    for i in range(3):
        fin=d2[i]#Otherwise error: float object is not subscriptable
        sta,current=dc[i],dc[i]
        n=1
        e=360/4096
        print("\n{0})".format(ang[i]))
        if sta<fin: #increase
            while fin-(sta+(n*e)) >= 0:
                current=sta+(n*e)
                #print(n,current)
                n = n+1 #NOTE: nth iteration means it will stop AFTER d2 is reached, so n-1 is used instead of n
            print("Final number of steps: ",n-1,"  Result: ",sta+((n-1)*e))
            print("Predicted Encoder(at N): ",ec[i]+n-1)
            print("N of steps -1:         ",n-2,"  Result: ",sta+((n-2)*e)) #one step less
            print("N of steps +1:         ",n,"  Result: ",sta+((n)*e)) #one step more
        if sta>fin:
            while fin-(sta-(n*e)) <= 0:
                current=sta-(n*e)
                #print(n,current)
                n = n+1 #NOTE: Iteration will stop AFTER d2 is reached, so n-1 is used instead of n
            print("Final number of steps: ",n-1,"  Result: ",sta-((n-1)*e))
            print("Predicted Encoder(at N): ",ec[i]-(n-1))#Is correct, dont doubt
            print("N of steps -1:         ",n-2,"  Result: ",sta-((n-2)*e)) #one step less
            print("N of steps +1:         ",n,"  Result: ",sta-((n)*e)) #one step more
                
ec=[100,100,100]
dc=[0,0,2]
d2=[3,-3,2.1]
DtoE(ec,dc,d2)


B)
Final number of steps:  34   Result:  2.98828125
Predicted Encoder(at N):  134
N of steps -1:          33   Result:  2.900390625
N of steps +1:          35   Result:  3.076171875

L)
Final number of steps:  34   Result:  -2.98828125
Predicted Encoder(at N):  66
N of steps -1:          33   Result:  -2.900390625
N of steps +1:          35   Result:  -3.076171875

R)
Final number of steps:  1   Result:  2.087890625
Predicted Encoder(at N):  101
N of steps -1:          0   Result:  2.0
N of steps +1:          2   Result:  2.17578125
